In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [69]:
import pandas as pd
import numpy as np
import requests
import json
import matplotlib.pyplot as plt
import seaborn as sns
import re
from bs4 import BeautifulSoup
import time
import random

In [11]:
url = "/content/drive/MyDrive/aiffel_final_project/data_renew/aiffel_book.json"

with open(url,"r",encoding="utf-8") as f:
    data = json.load(f)

In [12]:
data = pd.DataFrame(data)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 21 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ISBN         50000 non-null  object 
 1   ITEM_ID      50000 non-null  int64  
 2   BID          42652 non-null  float64
 3   GOODS_NO     7348 non-null   float64
 4   분류           46967 non-null  object 
 5   제목           49996 non-null  object 
 6   부제           33406 non-null  object 
 7   원제           4506 non-null   object 
 8   저자           50000 non-null  object 
 9   발행자          50000 non-null  object 
 10  발행일          50000 non-null  object 
 11  페이지          50000 non-null  int64  
 12  가격           50000 non-null  int64  
 13  표지           50000 non-null  object 
 14  간략소개         48590 non-null  object 
 15  책소개          47002 non-null  object 
 16  저자소개         37934 non-null  object 
 17  목차           45459 non-null  object 
 18  출판사리뷰        29156 non-null  object 
 19  INSE

In [ ]:
# 책 소개 없는 애들만
x = data.loc[data['책소개'].isna(), ['ISBN']]

In [ ]:
x

,ISBN
36,9788962515510
47,9791186492369
95,9788928518357
102,9791162431382
104,9788915001275
...,...
49938,9791169501262
49950,9788968472398
49962,9788956768861
49974,9791136426611


In [ ]:
x['ISBN']

,ISBN
36,9788962515510
47,9791186492369
95,9788928518357
102,9791162431382
104,9788915001275
...,...
49938,9791169501262
49950,9788968472398
49962,9788956768861
49974,9791136426611


In [ ]:
x.reset_index(drop=True,inplace=True)

In [ ]:
x['ISBN'] = x['ISBN'].astype('int64')
# 혹시 몰라서 int로. 32bit되려나

In [ ]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2998 entries, 0 to 2997
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   ISBN    2998 non-null   int64
dtypes: int64(1)
memory usage: 23.6 KB


In [ ]:
x

,ISBN
0,9788962515510
1,9791186492369
2,9788928518357
3,9791162431382
4,9788915001275
...,...
2993,9791169501262
2994,9788968472398
2995,9788956768861
2996,9791136426611


In [ ]:
def get_product_url_from_isbn(isbn):
    isbn_str = str(isbn)
    # ISBN을 URL의 keyword에 대입
    search_url = f"https://search.kyobobook.co.kr/search?keyword={isbn_str}&gbCode=TOT&target=total"
    response = requests.get(search_url)
    if response.status_code != 200:
        print(f"ISBN {isbn_str}: 상태 코드 {response.status_code}") # 에러날 시 에러코드 확인
        return None

    soup = BeautifulSoup(response.text, 'html.parser')

    # auto_overflow_wrap prod_name_group calss는 검색하는 ISBN이 없을 시 생성되지 않음 - 조건문으로 활용하여 None 반환
    container = soup.find("div", class_="auto_overflow_wrap prod_name_group")
    if container is None:
        print(f"ISBN {isbn_str}: 검색 결과 없음")
        return None

    # 생성된 페이지의 목표 태그(<a>---<href>)에서 URL만 추출 - 없으면 None 반환
    a_tag = container.find("a", href=True)
    if a_tag:
        product_url = a_tag.get('href')
        print(f"ISBN {isbn_str}: URL -> {product_url}")
        return product_url
    else:
        print(f"ISBN {isbn_str}: URL을 찾을 수 없음")
        return None

In [ ]:
x['url'] = x['ISBN'].apply(get_product_url_from_isbn)

ISBN 9788962515510: 상품 URL -> https://product.kyobobook.co.kr/detail/S000000969234
ISBN 9791186492369: 검색 결과 없음 (auto_overflow_wrap prod_name_group class 없음)
ISBN 9788928518357: 상품 URL -> https://product.kyobobook.co.kr/detail/S000201463394
ISBN 9791162431382: 상품 URL -> https://product.kyobobook.co.kr/detail/S000001812721
ISBN 9788915001275: 상품 URL -> https://product.kyobobook.co.kr/detail/S000202706008
ISBN 9788962397215: 상품 URL -> https://product.kyobobook.co.kr/detail/S000000966896
ISBN 9791137225138: 상품 URL -> https://product.kyobobook.co.kr/detail/S000060614084
ISBN 9788952785688: 상품 URL -> https://product.kyobobook.co.kr/detail/S000000734814
ISBN 9791137210981: 상품 URL -> https://product.kyobobook.co.kr/detail/S000060611981
ISBN 9791190145657: 상품 URL -> https://product.kyobobook.co.kr/detail/S000001936018
ISBN 9791127296377: 상품 URL -> https://product.kyobobook.co.kr/detail/S000060610785
ISBN 9791160455908: 검색 결과 없음 (auto_overflow_wrap prod_name_group class 없음)
ISBN 9791137267169: 

In [ ]:
file_path = "/content/drive/MyDrive/aiffel_final_project/data_renew/x_url.csv"

pd.DataFrame(x).to_csv(file_path, index=False)

In [ ]:
x['url'].isna().sum()

499

### 이미 존재하는 책 소개 row

In [55]:
description = pd.read_csv("/content/drive/MyDrive/aiffel_final_project/data_renew/book_description_from_naver.csv")
x = pd.read_csv("/content/drive/MyDrive/aiffel_final_project/data_renew/x_url.csv")

In [56]:
description

,ISBN,책소개
0,9791162431382,"임영석 시집 『나, 이제부터 삐딱하게 살기로 했다』는 크게 5부로 나누어져 있으며 ..."
1,9788962397215,▶ 이 책은 세법이론을 다룬 이론서입니다. 세법이론의 기초적이고 전반적인 내용을 학...
2,9788952785688,"엄마, 마녀가 나를 잡아먹으려고 해요! \n\n'네버랜드 마음이 자라는 성장 그림책..."
3,9791190145657,▶ 한국회화에 관한 내용을 담은 전문서적입니다.
4,9788982231032,"『무도와 무사도 인문학』은 〈일본 무도의 철학적 고찰〉, 〈무사의 마음, 일본의 국..."
...,...,...
868,9791168942882,말랑말랑 기분 좋은 치유계 EVERYDAY!\n\n가지런한 바가지 앞머리가 매력 포...
869,9791136412331,청라 퓨전 판타지 장편소설 『망작의 삼공자로 사는 법』 제3권. 북방의 군주라 불리...
870,9788942592791,▶ 이 책은 환경위해관리를 위한 노출평가를 다룬 이론서입니다. 환경위해관리를 위한 ...
871,9791137237254,태극권은 중국뿐만 아니라 전 세계적으로 많은 수련인구를 가지고 있다.\n\n수련생들...


In [57]:
description.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 873 entries, 0 to 872
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ISBN    873 non-null    int64 
 1   책소개     873 non-null    object
dtypes: int64(1), object(1)
memory usage: 13.8+ KB


In [58]:
x = x.merge(description[['ISBN', '책소개']], on='ISBN', how='left')

In [59]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2998 entries, 0 to 2997
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ISBN    2998 non-null   int64 
 1   url     2499 non-null   object
 2   책소개     873 non-null    object
dtypes: int64(1), object(2)
memory usage: 70.4+ KB


In [60]:
x['책소개'].isna().value_counts()

,count
책소개,
True,2125
False,873


In [61]:
data['출판사리뷰'].isna().value_counts()

,count
출판사리뷰,
False,29156
True,20844


In [62]:
data['ISBN'] = data['ISBN'].astype('int64')

In [63]:
x = x.merge(data[['ISBN','출판사리뷰']], on='ISBN', how='left')

In [64]:
x = x.merge(data[['ISBN','분류']], on='ISBN', how='left')

In [65]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 21 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ISBN         50000 non-null  int64  
 1   ITEM_ID      50000 non-null  int64  
 2   BID          42652 non-null  float64
 3   GOODS_NO     7348 non-null   float64
 4   분류           46967 non-null  object 
 5   제목           49996 non-null  object 
 6   부제           33406 non-null  object 
 7   원제           4506 non-null   object 
 8   저자           50000 non-null  object 
 9   발행자          50000 non-null  object 
 10  발행일          50000 non-null  object 
 11  페이지          50000 non-null  int64  
 12  가격           50000 non-null  int64  
 13  표지           50000 non-null  object 
 14  간략소개         48590 non-null  object 
 15  책소개          47002 non-null  object 
 16  저자소개         37934 non-null  object 
 17  목차           45459 non-null  object 
 18  출판사리뷰        29156 non-null  object 
 19  INSE

In [66]:
x = x.merge(data[['ISBN','목차']], on='ISBN', how='left')

In [67]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2998 entries, 0 to 2997
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ISBN    2998 non-null   int64 
 1   url     2499 non-null   object
 2   책소개     873 non-null    object
 3   출판사리뷰   352 non-null    object
 4   분류      2895 non-null   object
 5   목차      1714 non-null   object
dtypes: int64(1), object(5)
memory usage: 140.7+ KB


In [68]:
x['출판사리뷰'].isna().value_counts()

,count
출판사리뷰,
True,2646
False,352


In [70]:
!pip install selenium
!apt-get update
!apt-get install -y chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,341 kB]
Fetched 1,343 kB in 2s (625 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu

In [71]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import random

#### 분류

In [75]:
# Selenium 설정 (Colab 환경 등)
chrome_options = Options()
chrome_options.add_argument("--headless")           # 헤드리스 모드
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")

driver = webdriver.Chrome(options=chrome_options)

def extract_category(url):
    """
    주어진 URL에 대해 랜덤 딜레이 후 Selenium으로 상세페이지에 접속하고,
    BeautifulSoup을 사용해 카테고리 정보를 추출하여
    ">국내도서>시/에세이>한국시>현대시" 형태의 문자열을 반환합니다.
    """
    # 랜덤 딜레이 (3 ~ 7초)
    delay = random.uniform(3, 7)
    time.sleep(delay)

    driver.get(url)
    # 페이지 완전 로드를 위한 대기 (필요 시 조정)
    time.sleep(6)

    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    ul = soup.select_one('#scrollSpyProdInfo > div:nth-of-type(4) > div:nth-of-type(1) > ul')
    if ul:
        links = ul.select("a.intro_category_link")
        categories = [link.get_text(strip=True) for link in links]
        return ">" + ">".join(categories)
    else:
        return None

In [76]:
# x['url']가 존재하고 x['분류']가 null인 행들만 선택
mask = x['url'].notnull() & x['분류'].isnull()
target_indices = x[mask].index.tolist()

print(f"전체 처리 대상 개수: {len(target_indices)}")

# 한 번에 3개씩 배치 처리 혹시 모를 트래픽
batch_size = 3
total = len(target_indices)

for i in range(0, total, batch_size):
    batch_indices = target_indices[i:i+batch_size]
    print(f"배치 {i//batch_size+1} 처리 중 (총 {len(batch_indices)}개)...")
    for idx in batch_indices:
        url = x.loc[idx, 'url']
        category_info = extract_category(url)
        x.loc[idx, '분류'] = category_info
        print(f"인덱스 {idx} | URL: {url} -> 분류: {category_info}")
    print(f"배치 {i//batch_size+1} 처리 완료, 다음 배치 전 7초 대기...")
    time.sleep(7)

driver.quit()

전체 처리 대상 개수: 88
배치 1 처리 중 (총 3개)...
인덱스 16 | URL: https://product.kyobobook.co.kr/detail/S000061695386 -> 분류: >국내도서>취업/수험서>전문직자격증>행정사
인덱스 31 | URL: https://product.kyobobook.co.kr/detail/S000000902976 -> 분류: >국내도서>정치/사회>법학>소송/판례>민사소송(법)>국내도서>정치/사회>대학교재>법학>국내도서>대학교재>정치/사회/법>법학
인덱스 75 | URL: https://product.kyobobook.co.kr/detail/S000000902909 -> 분류: >국내도서>과학>교양과학>교양생물>생물이야기
배치 1 처리 완료, 다음 배치 전 7초 대기...
배치 2 처리 중 (총 3개)...
인덱스 131 | URL: https://product.kyobobook.co.kr/detail/S000061425529 -> 분류: >국내도서>취업/수험서>전문직자격증>변리사
인덱스 141 | URL: https://product.kyobobook.co.kr/detail/S000001764248 -> 분류: >국내도서>기술/공학>의학>보건학>보건의료법규>국내도서>기술/공학>대학교재>의학>국내도서>대학교재>기술공학>의학
인덱스 147 | URL: https://product.kyobobook.co.kr/detail/S000061694344 -> 분류: >국내도서>정치/사회>법학>상법>특허/상표/지식재산/저작권
배치 2 처리 완료, 다음 배치 전 7초 대기...
배치 3 처리 중 (총 3개)...
인덱스 151 | URL: https://product.kyobobook.co.kr/detail/S000201274208 -> 분류: >국내도서>경제/경영>세무/회계>양도/소득/재산세>국내도서>경제/경영>대학교재>국내도서>대학교재>경제/경영
인덱스 154 | URL: https://ebook-product.kyobobook

In [77]:
# 분류만 채움
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2998 entries, 0 to 2997
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ISBN    2998 non-null   int64 
 1   url     2499 non-null   object
 2   책소개     873 non-null    object
 3   출판사리뷰   352 non-null    object
 4   분류      2973 non-null   object
 5   목차      1714 non-null   object
dtypes: int64(1), object(5)
memory usage: 140.7+ KB


In [100]:
file_path = "/content/drive/MyDrive/aiffel_final_project/data_renew/x_url.csv"

pd.DataFrame(x).to_csv(file_path, index=False)

#### 책소개

In [88]:
# x DataFrame에 대해 x['url']가 존재하고 x['책소개']가 null인 행들만 선택
mask = x['url'].notnull() & x['책소개'].isnull()
target_indices = x[mask].index.tolist()

print(f"전체 처리 대상 개수: {len(target_indices)}")

# 한 번에 5개씩 배치 처리 (트래픽 부담 완화)
batch_size = 5
total = len(target_indices)

for i in range(0, total, batch_size):
    batch_indices = target_indices[i:i+batch_size]
    print(f"배치 {i//batch_size+1} 처리 중 (총 {len(batch_indices)}개)...")
    for idx in batch_indices:
        url = x.loc[idx, 'url']
        intro_text = extract_book_intro(url)
        x.loc[idx, '책소개'] = intro_text
        print(f"인덱스 {idx} | URL: {url} -> 책소개: {intro_text}")
    print(f"배치 {i//batch_size+1} 처리 완료, 다음 배치 전 7초 대기...")
    time.sleep(2)

driver.quit()

전체 처리 대상 개수: 1655
배치 1 처리 중 (총 5개)...
인덱스 0 | URL: https://product.kyobobook.co.kr/detail/S000000969234 -> 책소개: None
인덱스 6 | URL: https://product.kyobobook.co.kr/detail/S000060614084 -> 책소개: None
인덱스 8 | URL: https://product.kyobobook.co.kr/detail/S000060611981 -> 책소개: None
인덱스 10 | URL: https://product.kyobobook.co.kr/detail/S000060610785 -> 책소개: None
인덱스 12 | URL: https://product.kyobobook.co.kr/detail/S000060620237 -> 책소개: None
배치 1 처리 완료, 다음 배치 전 7초 대기...
배치 2 처리 중 (총 5개)...
인덱스 13 | URL: https://product.kyobobook.co.kr/detail/S000003945769 -> 책소개: None
인덱스 18 | URL: https://product.kyobobook.co.kr/detail/S000061442350 -> 책소개: None
인덱스 29 | URL: https://product.kyobobook.co.kr/detail/S000001977732 -> 책소개: None
인덱스 32 | URL: https://product.kyobobook.co.kr/detail/S000200489643 -> 책소개: None
인덱스 34 | URL: https://product.kyobobook.co.kr/detail/S000201045395 -> 책소개: None
배치 2 처리 완료, 다음 배치 전 7초 대기...
배치 3 처리 중 (총 5개)...
인덱스 38 | URL: https://product.kyobobook.co.kr/detail/S000201368469 

KeyboardInterrupt: 

In [98]:
import time
import random
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

# Selenium 설정 (Chrome headless 모드)
chrome_options = Options()
chrome_options.add_argument("--headless")           # 헤드리스 모드
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")
driver = webdriver.Chrome(options=chrome_options)

def extract_book_intro(url):
    """
    주어진 URL에 대해 랜덤 딜레이 후 Selenium으로 상세페이지에 접속하고,
    XPath 영역
       //*[@id="scrollSpyProdInfo"]/div[4]/div[2]
    와
       //*[@id="scrollSpyProdInfo"]/div[5]/div[2]
    내에서 클래스가 "info_text fw_bold" 또는 "info_text"인 모든 div 요소(또는 해당 요소 내의 텍스트 노드)를
    순차적으로 이어붙여 하나의 문자열로 반환합니다.

    만약 두 영역 모두에서 해당 클래스 요소가 전혀 없다면, None을 반환합니다.
    """
    # 랜덤 딜레이 (2 ~ 3초)
    time.sleep(random.uniform(1, 2))

    driver.get(url)
    # 페이지가 완전히 로드될 때까지 대기 (필요 시 조정)
    time.sleep(1)

    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    # 두 영역 컨테이너 선택
    containers = []
    container4 = soup.select_one('#scrollSpyProdInfo > div:nth-of-type(4) > div:nth-of-type(2)')
    container5 = soup.select_one('#scrollSpyProdInfo > div:nth-of-type(5) > div:nth-of-type(2)')
    if container4:
        containers.append(container4)
    if container5:
        containers.append(container5)

    # 두 영역에서 "info_text" 관련 요소(예: "info_text fw_bold" 또는 "info_text")의 텍스트 노드 추출
    texts = []
    for cont in containers:
        divs = cont.find_all("div", class_=lambda c: c and "info_text" in c)
        if divs:
            for div in divs:
                texts.extend(list(div.stripped_strings))

    if texts:
        return " ".join(texts)
    else:
        return None

In [99]:
# 재실행 시작 인덱스 지정 (예: 10부터 처리)
start_index = 217

# x DataFrame에 대해 x['url']가 존재하고 x['책소개']가 null인 행들 중에서 인덱스가 start_index 이상인 경우 선택
mask = x['url'].notnull() & x['책소개'].isnull() & (x.index >= start_index)
target_indices = x[mask].index.tolist()

print(f"전체 처리 대상 개수 (인덱스 {start_index}부터): {len(target_indices)}")

# 한 번에 5개씩 배치 처리 (트래픽 부담 완화)
batch_size = 10
total = len(target_indices)

for i in range(0, total, batch_size):
    batch_indices = target_indices[i:i+batch_size]
    print(f"배치 {i//batch_size+1} 처리 중 (총 {len(batch_indices)}개)...")
    for idx in batch_indices:
        url = x.loc[idx, 'url']
        intro_text = extract_book_intro(url)
        x.loc[idx, '책소개'] = intro_text
        print(f"인덱스 {idx} | URL: {url} -> 책소개: {intro_text}")
    print(f"배치 {i//batch_size+1} 처리 완료, 다음 배치 전 7초 대기...")
    time.sleep(1)

driver.quit()

전체 처리 대상 개수 (인덱스 217부터): 1535
배치 1 처리 중 (총 10개)...
인덱스 217 | URL: https://product.kyobobook.co.kr/detail/S000200551569 -> 책소개: 2020년 초부터 전 세계적으로 확산된 코로나는 우리나라 정치, 경제, 사회, 문화 등 모든 분야에 영향을 미쳤다. 거리두기 영향과 대인접촉 기피현상으로 재택근무가 늘어나고, 각종 모임과 행사 및 회식이 줄어드는 등 큰 사회적 변화를 겪었다. 코로나 영향으로 부동산 시장도 큰 영향을 받았다.
인덱스 219 | URL: https://product.kyobobook.co.kr/detail/S000201408366 -> 책소개: None
인덱스 220 | URL: https://product.kyobobook.co.kr/detail/S000201054243 -> 책소개: 이 책은 사회복지학을 다룬 이론서이다. 알면서도 몰랐던 장애의 기초적이고 전반적인 내용을 학습할 수 있도록 구성하였다.
인덱스 228 | URL: https://product.kyobobook.co.kr/detail/S000200152387 -> 책소개: None
인덱스 229 | URL: https://product.kyobobook.co.kr/detail/S000200367257 -> 책소개: 독자대상 : 경찰공무원 시험 준비생 구성 : 이론
인덱스 230 | URL: https://product.kyobobook.co.kr/detail/S000061695353 -> 책소개: 이 책은 소박한 민속과 민가의 향기 3를 다룬 한옥건축서적이다. 소박한 민속과 민가의 향기 3에 관한 기초적이고 전반적인 내용들이 수록되어 있다.
인덱스 236 | URL: https://product.kyobobook.co.kr/detail/S000001984653 -> 책소개: 『어도비 포토샵CC』는 〈포토샵 튜닝〉, 〈환경설정〉, 〈스크레치 디스크〉, 〈사진 촬영 기법과 포토샵〉, 〈이미지 해

In [ ]:
# https://product.kyobobook.co.kr/detail/S000200419232
# https://product.kyobobook.co.kr/detail/S000200497027
# https://product.kyobobook.co.kr/detail/S000201040684
# https://product.kyobobook.co.kr/detail/S000200417698

# https://product.kyobobook.co.kr/detail/S000061777727

In [93]:
naru = pd.read_csv("/content/drive/MyDrive/aiffel_final_project/data_renew/book_description_from_naru.csv")

In [101]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2998 entries, 0 to 2997
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ISBN    2998 non-null   int64 
 1   url     2499 non-null   object
 2   책소개     2240 non-null   object
 3   출판사리뷰   352 non-null    object
 4   분류      2973 non-null   object
 5   목차      1714 non-null   object
dtypes: int64(1), object(5)
memory usage: 140.7+ KB


In [92]:
file_path = "/content/drive/MyDrive/aiffel_final_project/data_renew/x_url.csv"

pd.DataFrame(x).to_csv(file_path, index=False)

In [31]:
# original
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2998 entries, 0 to 2997
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ISBN    2998 non-null   int64 
 1   url     2499 non-null   object
 2   책소개     873 non-null    object
 3   출판사리뷰   352 non-null    object
 4   분류      2895 non-null   object
 5   목차      1714 non-null   object
dtypes: int64(1), object(5)
memory usage: 140.7+ KB


* //*[@id="scrollSpyProdInfo"]/div[4]/div[3]/div/text()[1]
    * 해당 주소로 758개 다시 받아오기
* 출판사리뷰(서평) + 추천사
    * 존재하는 것만 빨리빨리 보고 넘어가게 코드 짜기
* 목차